In [15]:
# -*- encoding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [16]:
import numpy as np

from keras.layers import Activation,Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional,Dropout,GRU

from keras.models import Sequential,load_model

from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split

import collections  #用来统计词频
import logging
import pandas as pd

In [17]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [18]:
# 训练模型
def trainMdl_title(mtype,matFile,mdlFile,maxfeat):
    
    # 选入词典的词条数目
    MAX_FEATURES = maxfeat
        
    # 选入词典的词条数目 + 2 （pad + UNK）
    vocab_size = MAX_FEATURES + 2
    
    # 装载数据矩阵
    dataMat = np.load(matFile)
    print "matrix shape:",dataMat.shape
    
    # 取出训练数据
    X =dataMat[:50000,:-1]
    print "X shape:",X.shape
    
    MAX_SENTENCE_LENGTH = X.shape[1]
    print "MAX_SENTENCE_LENGTH:",MAX_SENTENCE_LENGTH
    
    # 取出标签，保存为向量
    y = dataMat[:50000,-1]
    print "y shape:",y.shape
    
    # 划分数据
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 把一维的标签做onehot，pd.get_dummies的结果是df,把df转为ndarray (as_matrix())
    ytrain_label = pd.get_dummies(ytrain).as_matrix()
    ytest_label = pd.get_dummies(ytest).as_matrix()
    
#     ytrain_label = ytrain
#     ytest_label = ytest
    
    print('shape of train label:',ytrain_label.shape)  
    print('shape of test label:',ytest_label.shape)
    
   
    if mtype == 'lstm':
        # 构建网络
        HIDDEN_LAYER_SIZE = 32
        EMBEDDING_SIZE = 128
        
        model = Sequential()
        model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
        model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.5, recurrent_dropout=0.5))
        model.add(Dense(2,activation='softmax'))
        model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
    elif mtype == 'bilstm':
        # 构建网络
        HIDDEN_LAYER_SIZE = 32
        EMBEDDING_SIZE = 128
        
        model = Sequential()
        model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
        model.add(Bidirectional(GRU(HIDDEN_LAYER_SIZE, dropout=0.5, recurrent_dropout=0.5)))
        model.add(Dense(2, activation='softmax'))
        model.compile('rmsprop', 'categorical_crossentropy', metrics=['accuracy'])
    elif mtype == 'GRU':
        # 构建网络
        HIDDEN_LAYER_SIZE = 32
        EMBEDDING_SIZE = 128

        model = Sequential()
        model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))
        model.add(GRU(HIDDEN_LAYER_SIZE, dropout=0.5, recurrent_dropout=0.5))
        model.add(Dense(2,activation='softmax'))
        # ----
        model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
    
    # 训练网络
    BATCH_SIZE = 128
    NUM_EPOCHS = 10
    
#     if mtype == 'bilstm':
#         from keras.utils.np_utils import to_categorical
#         ytrain = to_categorical(ytrain)
#         ytest = to_categorical(ytest)
    # lstm
    model.fit(Xtrain, ytrain_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest_label))
    
    # bilstm
#     model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,validation_data=(Xtest, ytest))
    
    # 预测 
    score, acc = model.evaluate(Xtest, ytest_label, batch_size=BATCH_SIZE)
    print("\nTest score: %.3f, accuracy: %.3f" % (score, acc))
   
    # 模型持久化
    model.save(mdlFile) 
    

In [19]:
def testModel(inFile,testFile,testTxt,mdlFile):
      
    # 装载模型
    model = load_model(mdlFile)
    
    testLabel,testText = getTestSet(testTxt)
    
    # 预测 
    print('-------------------------------------------')
    print('{}     |   {}  |    {}'.format('AI評価','人の評価','評価原文'))
    print('-------------------------------------------')
    for i in range(3):
        xtest = X[i].reshape(1,100)
        ylabel = y[i]
        pred = model.predict(xtest)
        # 打印预测出的概率
#         print pred
        ypred = getLabel(pred[0].argmax())
        sent = " ".join([index2word[x] for x in xtest[0] if x != 0])
        print('{}   |  {}   |  {}'.format(getString(ypred), getString(str(ylabel)), testText[i]))
        print('-------------------------------------------')

In [ ]:
def main():
    
#     inFile = "./data/test.txt"
    mdlPath = "/home/hadoop/DataSencise/bdci2017/BDCI2017-360/model/"
    
    matFile_title = mdlPath + "train_matrix_title.npy"
    matFile_content = mdlPath + "matrix_content.npy"
    
    lstmMdl = mdlPath + "lstm_title.h5"
    bilstmMdl = mdlPath + "lstm_content.h5"
    GRUMdl = mdlPath + "GRU_title.h5"
   
    #
    MAX_FEATURES = 310000
    MAX_SENTENCE_LENGTH = 20
        
    # 训练    
#     trainMdl_title('lstm',matFile_title,lstmMdl,MAX_FEATURES)
    trainMdl_title('GRU',matFile_title,GRUMdl,MAX_FEATURES)


In [ ]:
if __name__ == '__main__':
    main()